In [1]:
import tensorflow as tf
import numpy as np
import zipfile
import librosa
import sounddevice as sd

# Download the model to yamnet-classification.tflite
interpreter = tf.lite.Interpreter("/content/lite-model_yamnet_classification_tflite_1.tflite")

input_details = interpreter.get_input_details()
waveform_input_index = input_details[0]['index']
output_details = interpreter.get_output_details()
scores_output_index = output_details[0]['index']

# Load the label names
labels_file = open('/content/labels.txt').readlines()
labels = [l.strip() for l in labels_file]

# Define a function to process audio in real-time
def process_audio(samples, sample_rate):
    # Resample audio to 16kHz
    y_resampled = librosa.resample(samples, orig_sr=sample_rate, target_sr=16000)
    # Normalize audio waveform
    y_norm = librosa.util.normalize(y_resampled)
    
    interpreter.resize_tensor_input(waveform_input_index, [y_norm.size], strict=False)
    interpreter.allocate_tensors()
    interpreter.set_tensor(waveform_input_index, y_norm)
    interpreter.invoke()
    scores = interpreter.get_tensor(scores_output_index)

    # Get the top predicted class
    top_class_index = scores.argmax()

    # Print the predicted class label
    print(labels[top_class_index])

# Define the audio callback function
def audio_callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    process_audio(indata[:, 0], sd.default.samplerate)

# Start the audio stream
with sd.InputStream(callback=audio_callback):
    print("Yamnet is listening...")
    sd.sleep(100000)


ModuleNotFoundError: No module named 'tensorflow'